In [1]:
text = """The expansion of online social networking has given citizens unprecedented ability to share information, create communities and organize social movements. From a governmental perspective, this could be quite a challenge as social media may be used as a platform to directly challenge the government. It is also a ‘Wild West’ in which misinformation could spread and hatred incited quickly with little means for regulation. Currently, while there are measures to mitigate the challenges posed by social media, they are still in their infancy and likely, very limited in their effectiveness. At the same time, the benefits of social media hardly offset the challenges, and hence it is reasonable to argue that social media poses more of a challenge to governments than not.

The widespread reach and unregulated nature of social media is perhaps the biggest reason governments find it a challenge. According to the International Communication Union, there are more than 3 billion Internet users worldwide. Most will have access to social media of some sort—such as Facebook, Instagram, Twitter, YouTube and LinkedIn. This means that large segments of the population could be influenced by the use of social media for communication, either directly or indirectly. These platforms are difficult to regulate as there are very few existing laws governing social media, and it is difficult to enforce rulings when the platform’s owners are based in a different country. The sheer volume of information on those platforms also makes it impossible for regulators to keep close watch on everything. The most famous use of social media to challenge governmental authority is perhaps the Arab Spring of 2011, where social media was used to rapidly spread awareness of and coordinate the protests. The uprising led to the overthrow of Ben Ali, Mubarak and Gaddafi in Tunisia, Egypt and Libya respectively, and civil war in most others. Notwithstanding the fact that those governments were oppressive and an uprising may perhaps be justified, the fact remains that the use of social media could turn the people against their leaders with unprecedented intensity and speed. Similarly, in China, Facebook was used during the July 2009 Urumqi Riots by the minority Muslims in Xinjiang which led to the deaths of almost 200 people and almost 2,000 injured, in turn leading to the ban of the social media platform nation-wide. However, even on alternate, highly-censored platforms, many users have found sophisticated ways of going around censorship and regulation—the use of Virtual Private Network (VPN) technology allows users to access apps and websites which are officially banned, and the creative use of words and phrases have been able to circumvent regulation. For instance, netizens in China use “May 35th1989” to refer to the date Tiananmen Incident—“July 4th2019”—which itself is a banned phrase as the incident remains taboo. For governments who want to preserve stability and control over its citizens, social media poses a serious challenge as a means in which uprisings could be quickly and widely coordinated and gain a high public profile.

Even when social media use does not lead to mass revolt, it could still pose a challenge as an arena in which dangerous ideas circulate. Terrorist groups such as ISIS regularly upload threatening videos and propaganda messages on social media, leading to ‘self-radicalisation’ among citizens abroad. Foreign governments may also use social media to interfere in the affairs of another nation—a gross violation of the norms of international conduct, where it is generally agreed that countries would not intervene in each other’s domestic affairs. In August 2019, the New York Timesreported that China was using LinkedIn to recruit spies in the US, by posing as businessmen or academics inviting American targets to travel to China for an academic conference or presentation, or a business opportunity. From there, the Chinese agents start building a rapport with their targets and slowly convince them to hand over state secrets. The US is also a victim of Russia’s use of Facebook to interfere in the 2016 Presidential Election that saw the election of Donald Trump—at the time perceived by Russia as their preferred candidate. Exacerbating these problems is the proliferation of “fake news”, which many experts suspect not only influenced the outcome of the 2016 US Presidential Election, but the 2016 Brexit vote in the UK as well. In Singapore, the government passed a new piece of legislation in 2019—the Protection from Online Falsehoods and Manipulation Act—proving that social media poses a serious enough challenge to governments hoping to protect their citizens from malicious content. Additionally, profit-oriented social media companies have a stronger interest in earning profits than adopting responsible behaviour. For instance, users are particularly vulnerable to confirmation bias as online algorithms detect users’ personal opinions based on their online behaviour, and upload relevant posts on their news feed accordingly. Since most people currently receive their daily news from social media platforms, this means that society may become more polarised. Therefore, the unregulated but highly populated ‘Wild West’ of social media poses a huge, and likely difficult to solve, challenge for governments as dangerous information circulate widely and quickly among their citizens.

The challenges posed by social media to governments may be negligible if they provide them with substantial benefits; however, that is unlikely to be the case. Politicians may find social media useful in helping them directly communicate with their voters. This makes leaders closer to the ground and more relatable. An extreme case happened in Turkey in 2016, when President Recep Tayyip Erdogan managed to survive a military coup by appealing to his supporters via a live stream on FaceTime. More typically, politicians use social media to upload photos of milestone events, inform followers of upcoming events, and have a direct line of communication with the public. However, their actions are also heavily scrutinised—any misstep on social media, no matter how slight, could cost them politically. Social media now acts as a check and balance against the government—something like a ‘Fifth Estate’ or watchdog exposing governmental wrongdoings. For instance, in 2011, American congressman Anthony Weiner was forced to resign after a sexually suggestive photo he sent to a woman via Twitter was leaked to the public. With the ease in which misinformation can be spread through fabricated or outdated photos, or situations taken out of context, even the most honest and upright government would likely hold some reservations over the positive aspects of social media. Ultimately, most governments would conclude that social media ultimately poses a net challenge, and is largely a necessary evil that has to be managed.

In summary, social media is largely a challenge to government as a widely used by highly unregulated platform in which forces of disruption could spread like wildfire. Citizens may be driven to rise up against their leaders, or have their views distorted by outside propaganda. Such problems are threats to national security, and despite the benefits social media brings to governments in connecting with the people, it is safe to conclude that most governments rightly perceive social media as largely a threat that produces only some positive side effects."""

In [14]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments
from transformers import PreTrainedModel
from transformers.pipelines.pt_utils import KeyDataset

from tqdm.auto import tqdm

import functools

from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score

import pandas as pd
import numpy as np
import logging
from glob import glob
from os import path

from IPython.display import HTML, display

import torch

from spacy import displacy

In [5]:
from lib.model import *

In [3]:
model_checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [29]:
model_path = r"models_gitignored/roberta_base/checkpoint-75756"
teacher = AutoModelForSequenceClassification.from_pretrained(model_path, id2label=category_codes)

In [30]:
import sys
sys.getsizeof(teacher)

48

In [31]:
model_path = r"models_gitignored\roberta_noisy_iterations\iter1\checkpoint-65255"
student = AutoModelForSequenceClassification.from_pretrained(model_path, id2label=category_codes)

In [32]:
sys.getsizeof(student)

48

In [24]:
def displayAnnots(text, model=student, title=""):
    df = pseudoLabel(pd.Series([text]), tokenizer, student)[["text", "category"]]
    ents = functools.reduce(lambda lst, new: lst+[lst[-1]+len(new)], df.text, [0])
    df["start"] = ents[:-1]
    df["end"] = ents[1:]
    doc = {
        "text": df.text.sum(),
        "ents": [dict(start=int(row["start"]), end=int(row["end"]), label=row["category"]) for i, row in df.iterrows()],
        "title": title
    }

    labels = ['Lead', 'Position', 'Evidence', 'Claim', 'Concluding Statement', 'Counterclaim', 'Rebuttal']
    colors = ["#ACDDDE", "#CAF1DE", "#E1F8DC", "#FEF8DD", "#FFE7C7", "#F7D8BA", "#D6CDEA"]
    options = {"ents": labels, "colors": dict(zip(labels, colors))}
    displacy.render(doc, style="ent", options=options, manual=True, jupyter=True)
    print('\n\n')
    return df



meta = displayAnnots(text, title="To what extent does social media pose a challenge for the government?")

In [25]:
meta = displayAnnots(text, teacher, title="To what extent does social media pose a challenge for the government?")

In [19]:
text2 = """Mankind has always had a knack for finding the easy way around the most difficult tasks. Once we developed the technology needed to survive the brutal forces of nature in the Stone Age, mankind embarked on a journey of technological advancement in pursuit of a better life. Ironically, in our quest to automate and optimize our processes, we have become heavily reliant on technology, rendering many skills that were once considered fundamental and essential useless. Therefore, I believe that technology, while greatly improving our quality of life, has had an adverse impact on the skill levels of mankind.

The negative impact that technology has had on our interpersonal skills is perhaps the most pervasive and insidious one yet. Technological advancements in communication – more particularly the development of the smartphone – has revolutionised the way we interact. Texts and direct messages have now substituted face-to-face interaction, and emojis and stickers are at present the most convenient conveyors of our mood. With the younger generation of today growing up in a world like this, many of them are more comfortable with socializing online than in real life. Undeniably, this new form of communication has many perks: speed, immediacy, and the ability to avoid direct confrontation are just a few. However, this has led to many of them losing their ability to communicate effectively with others. They shy away from situations that require them to mingle, choosing to use social media to get to know others instead. They are better able to sustain a conversation online than offline, to read text cues than a change in tone, to interpret the blue tick than body language. This dependence has resulted in a situation where many use their smartphones even in the presence of others, to the detriment of their communication skills. Naturalistic research indicates that people who had conversations without the presence of a mobile phone felt that these conversations felt higher levels of ‘empathetic concern’, as opposed to those who conversed with the presence of a mobile phone. This suggests that the very act of using technology to cope with social realities and communication impedes us from fully developing our skills of empathy and interpersonal communication. The multi-faceted art of communication has been relegated to the mechanical act of typing and tapping – very sorry degeneration indeed.

Furthermore, technological advancements have witnessed the degradation of the skills people require for their jobs. Most jobs today are heavily reliant on information technology as it provides easier ways around otherwise labour intensive, manual jobs. Machines have replaced manual labour at the frontlines of production, and computer applications have created easier ways of accessing and handling data. This has thus seen a colossal upheaval in the way people work – they are now dependent on computer applications and software to do the work that previously required a high level of skill to do. Programs such as Excel organize, tabulate, and analyse data for the user, making the accountant’s job much easier. Artificial intelligence software, such as IBM’s Watson, uses natural language processing to interpret chunks of unstructured data, allowing businessmen to derive meaning from the statistics that they have. In Singapore, Lex Quanta, a data analytics firm, has created an application to predict the division of assets in divorce cases, rendering the traditional skills which junior lawyers possess obsolete. Therefore, with the advancement of technology to the point that we are able to delegate half our workload to machines, the de-skilling of the workforce has occurred, negatively impacting the skills levels of many workers in various industries.

Additionally, our motor skills have also deteriorated greatly due to the advancements in technology. The widespread proliferation of smartphones and gaming consoles like Xboxes and Wiis has resulted in and more and more youngsters today preferring to stay indoors and indulging in those games instead of going outdoors and playing actual games. Additionally, video games have the added edge of being more easily accessible than engaging in sporting activities is, and hence, many youths spend much of their free time playing those games instead, growing better at them while leaving themselves completely unexposed to outdoor physical activity. Furthermore, the likelihood of a child getting exposed to video games than to sports at an earlier age is higher, for today’s cautious parents are more willing to have their children nestled within the confines of one’s home, transfixed by the dancing graphics on a screen, rather than being out kicking a ball around. Therefore, the advancements in technology have made it easier for youths of today to dedicate more time to their gaming skills rather than to their sporting abilities, leaving these motor skills relatively untouched and undeveloped, if not actively degenerating.

However, it must be noted that technology has had a positive impact on stimulating the minds of people at large, promoting critical thinking skills and creativity. With technological advancements, the average person in a developed country is now able to access an entire treasure trove of knowledge that has been collated online. He is now empowered to contribute solutions related to these areas of interest. Furthermore, the differing points of view and contrasting schools of thought that are readily available online allows one to broaden his or her perspectives, equipping one with the mental dexterity to think laterally, instead of just vertically. Technological advancements have also made an array of software applications and programs accessible to laymen, such as blogs, computer-aided design applications, and video and photo editing tools. These programs have allowed people to harness and display their creativity in a wide variety of ways – blogs have allowed people the liberty to write or make videos on any topic they like, computer-aided design programs have allowed people the ability to come up with design ideas for buildings and products, and video and photo editing software have allowed people to tap on their visual creativity to make their images more aesthetically pleasing. Therefore, with the smorgasbord of applications made available to all due to the technology of today, people are better able to think creatively and express themselves in such a way.

It can also be argued that technology has also improved our cognitive skills significantly. The advent of wiki pages, online learning platforms, as well as podcasts has equipped people today with the ability to learn about anything, anywhere, at any time. If one has a query about any topic, she can simply search it up online and find answers that would help her resolve their doubts. Learning is no longer confined to within the constraints of the classroom, and students today are urged to venture out of what is merely in the curriculum and do more independent research. Online sources are now acknowledged as legitimate references, underscoring their reliability and quality. Additionally, as there is so much information online, the act of thinking deeper is thus facilitated. People are more prone to deliberating on topics and asking more questions when they can find answers to their queries, and the Internet provides a ready bank of information to answer these queries.

All in all, it is no secret that technology has an impact on the skills of people today. While it is true that as mankind embraces the more efficient, more productive methods of doing things, it has facilitated the greater development of our creativity and intellect, the gains that we have made in that aspect have come at the cost of our soft skills. However, it is not too late to reverse this impact that technology has made on those skill sets of ours, but we have to first acknowledge the effects that it has had on us."""

meta = displayAnnots(text, "To what extent has technology had a negative impact on the skill levels of people?")

In [20]:
displayAnnots("Although the history of human invention spans over millennia, there have been relatively few game-changing technologies. From the manipulation of fire back in the primal age to the invention of the electronic transistor in the 1940s, these monumental developments have often been followed up by fervent exploration and a burgeoning of new technologies that stem from that initial spark.", "Leemen Chan et al")

,text,category,start,end
0,Although the history of human invention spans ...,Claim,0,386
0,Although the history of human invention spans ...,Evidence,386,772


In [21]:
displayAnnots(
    """It all started with Six Degrees. Launched in 1997, this website enabled users to upload a profile and make friends with other users. It was the predecessor of the many social networking sites to come, and its creation heralded the advent of a new era – an era in which social media would emerge as a new medium of communication. Defined as a group of Internet-based applications that allow the creation and exchange of user-generated content, social media takes on many different forms, from social networks to social news to even Wikis. Given how pervasive social media has become, it is indisputable that it can influence individuals, and hence wields the power to both divide and unite the community. While it has the potential to create rifts within society by perpetuating existing divides like the generational divide and creating echo chambers, it has an even greater potential to bring individuals together. It cuts across physical boundaries to unite, facilitates the sharing of different opinions, and sheds light on various otherwise unknown situation. Therefore, social media does not, in fact divide more than unite.

Admittedly, social media does perpetuate and exacerbate the generational gap between the older and the younger generations of today. The younger generation are known as ‘digital natives’ who have grown up with technological gadgets as their toys and social media as a legitimate means of self-expression. In fact, according to a research conducted by Pew Research Centre on 700 American youth in 2018, close to 95% of teenagers have access to a smartphone, and 45% say that they are online on social media platforms on a ‘almost constant’ basis. The older generation is remarkably less adept at using technology and social media: while 90% of those aged 18-29 use at least one social media account, only 40% of those aged above 65 do. With the older generations lacking an interest in and knowledge of social media, the two groups are unable to relate to each other, especially since one is so immersed in social media culture, such as memes, and the latter is not. This thereby creates an ever-widening chasm between the two of them.

Additionally, social media platforms have created filter bubbles, and their pervasive influence create echo chambers, where users are inundated with information that support their existing views and are underexposed to information that challenges their assumptions, thereby effectively isolating them in their own cultural or ideological bubbles. Through the use of algorithms, social media sites such as Google, Facebook, Instagram and Twitter have taken to doing this, streamlining the information their users see first based on their online behaviour. According to research by the Pew Research Centre, 47% of conservatives are likely to see Facebook posts aligned with their own views, as opposed to half that percentage for those who seek the middle ground. This form of filtering is rather worrisome because it affirms its users’ confirmation bias, creating the impression that only one correct stance – theirs – exists. It is not conducive for discourse since individuals are not exposed to other narratives which may challenge their own. Therefore, the evolution of such filter bubbles on social media may lead to the polarisation of various groups within the society, hence diving instead of uniting.

However, social media’s ability to unite should not be overlooked. There is no doubt that social media facilitates the interaction of people across borders. Social media allows people to make new friends from different nationalities, as well as sustain relationships with friends and family who have moved overseas. For example, through sites like Tumblr blogs, friendships can be forged between someone living in Asia and another living in Canada. These allow for a deeper understanding of other cultures, practices and ways of thinking that we may not otherwise be exposed to. With this greater comprehension comes more tolerance, and therefore greater unity on a global level. Additionally, social networking applications such as Skype and Facebook allow video calls to be made, which are helpful in maintaining that human touch. Social media has shrunk the world into one that can be accessed from the small screen of a smartphone, reducing the distance between people.

Next, social media allows individuals from all walks of life to air their views, and facilitates discourse between individuals with distinct, unique perspectives. Since close to 1.7 billion people of the 7.2 billion people in the world today have active social media accounts, there is a smorgasbord of differing opinions online. Social media platforms empower these individuals to broadcast their views, and thus provides one with a unique insight into the views of others. Take for example, Singapore’s political scene. Cognizant of the increasing online presence of Singaporeans, the government created a website called eGov2015 in a bid to ensure that feedback from Singaporeans from all walks of life can be heard, and to facilitate greater co-creation and collaboration between the government and the people. Social media therefore makes it easier for individuals and organizations alike to hear different views and thus act accordingly.

Finally, social media also helps to unite the global community by shedding light on situations all around the world. With the ease of dissemination of information, netizens all around the world are not just able to glean insights into the struggles of others but are empowered to take action. For example, the ALS Ice Bucket Challenge, started in the summer of 2014, raised $11.5 million for its cause. Over 17 million people uploaded their challenge videos to Facebook, and thus the cause was also championed by celebrities such as Taylor Swift, Bill Gates and Mark Zuckerberg. The challenge was started in a bid to raise awareness for the disease known as amyotrophic lateral sclerosis, and the use of social media spreading this challenge resulted in its resounding success. A more recent example would be the Trashtag Challenge, where social media users pick up litter, clean their environment and later post a photo of their accomplishments on social media. It has allowed users a better insight of the sheer scale of the plastics pollution problem. It is a good example of how social media can go further than raising awareness to mobilizing individuals to act, producing tangible results. In this way, social media allows for the creation of unity among people, for it raises awareness about issues that society is confronted with and mobilizes people to act in the interest of their communities.

In essence, while social media may cause rifts to occur and widen within society, its tendency to unite and bring people together is even greater. The availability of information online allows for the forging of a more understanding and hence, more united world and is something that would have been unattainable in a world fragmented into isolated groups.""",
    "“Social Media divides more than it unites.” To what extent is this true?"
)

,text,category,start,end
0,It all started with Six Degrees. Launched in 1...,Claim,0,7058
0,It all started with Six Degrees. Launched in 1...,Claim,7058,14116
0,It all started with Six Degrees. Launched in 1...,Claim,14116,21174
0,It all started with Six Degrees. Launched in 1...,Claim,21174,28232
0,It all started with Six Degrees. Launched in 1...,Claim,28232,35290
0,It all started with Six Degrees. Launched in 1...,Claim,35290,42348
0,It all started with Six Degrees. Launched in 1...,Claim,42348,49406
0,It all started with Six Degrees. Launched in 1...,Claim,49406,56464
0,It all started with Six Degrees. Launched in 1...,Claim,56464,63522
0,It all started with Six Degrees. Launched in 1...,Claim,63522,70580


In [26]:
displayAnnots(
    """It all started with Six Degrees. Launched in 1997, this website enabled users to upload a profile and make friends with other users. It was the predecessor of the many social networking sites to come, and its creation heralded the advent of a new era – an era in which social media would emerge as a new medium of communication. Defined as a group of Internet-based applications that allow the creation and exchange of user-generated content, social media takes on many different forms, from social networks to social news to even Wikis. Given how pervasive social media has become, it is indisputable that it can influence individuals, and hence wields the power to both divide and unite the community. While it has the potential to create rifts within society by perpetuating existing divides like the generational divide and creating echo chambers, it has an even greater potential to bring individuals together. It cuts across physical boundaries to unite, facilitates the sharing of different opinions, and sheds light on various otherwise unknown situation. Therefore, social media does not, in fact divide more than unite.

Admittedly, social media does perpetuate and exacerbate the generational gap between the older and the younger generations of today. The younger generation are known as ‘digital natives’ who have grown up with technological gadgets as their toys and social media as a legitimate means of self-expression. In fact, according to a research conducted by Pew Research Centre on 700 American youth in 2018, close to 95% of teenagers have access to a smartphone, and 45% say that they are online on social media platforms on a ‘almost constant’ basis. The older generation is remarkably less adept at using technology and social media: while 90% of those aged 18-29 use at least one social media account, only 40% of those aged above 65 do. With the older generations lacking an interest in and knowledge of social media, the two groups are unable to relate to each other, especially since one is so immersed in social media culture, such as memes, and the latter is not. This thereby creates an ever-widening chasm between the two of them.

Additionally, social media platforms have created filter bubbles, and their pervasive influence create echo chambers, where users are inundated with information that support their existing views and are underexposed to information that challenges their assumptions, thereby effectively isolating them in their own cultural or ideological bubbles. Through the use of algorithms, social media sites such as Google, Facebook, Instagram and Twitter have taken to doing this, streamlining the information their users see first based on their online behaviour. According to research by the Pew Research Centre, 47% of conservatives are likely to see Facebook posts aligned with their own views, as opposed to half that percentage for those who seek the middle ground. This form of filtering is rather worrisome because it affirms its users’ confirmation bias, creating the impression that only one correct stance – theirs – exists. It is not conducive for discourse since individuals are not exposed to other narratives which may challenge their own. Therefore, the evolution of such filter bubbles on social media may lead to the polarisation of various groups within the society, hence diving instead of uniting.

However, social media’s ability to unite should not be overlooked. There is no doubt that social media facilitates the interaction of people across borders. Social media allows people to make new friends from different nationalities, as well as sustain relationships with friends and family who have moved overseas. For example, through sites like Tumblr blogs, friendships can be forged between someone living in Asia and another living in Canada. These allow for a deeper understanding of other cultures, practices and ways of thinking that we may not otherwise be exposed to. With this greater comprehension comes more tolerance, and therefore greater unity on a global level. Additionally, social networking applications such as Skype and Facebook allow video calls to be made, which are helpful in maintaining that human touch. Social media has shrunk the world into one that can be accessed from the small screen of a smartphone, reducing the distance between people.

Next, social media allows individuals from all walks of life to air their views, and facilitates discourse between individuals with distinct, unique perspectives. Since close to 1.7 billion people of the 7.2 billion people in the world today have active social media accounts, there is a smorgasbord of differing opinions online. Social media platforms empower these individuals to broadcast their views, and thus provides one with a unique insight into the views of others. Take for example, Singapore’s political scene. Cognizant of the increasing online presence of Singaporeans, the government created a website called eGov2015 in a bid to ensure that feedback from Singaporeans from all walks of life can be heard, and to facilitate greater co-creation and collaboration between the government and the people. Social media therefore makes it easier for individuals and organizations alike to hear different views and thus act accordingly.

Finally, social media also helps to unite the global community by shedding light on situations all around the world. With the ease of dissemination of information, netizens all around the world are not just able to glean insights into the struggles of others but are empowered to take action. For example, the ALS Ice Bucket Challenge, started in the summer of 2014, raised $11.5 million for its cause. Over 17 million people uploaded their challenge videos to Facebook, and thus the cause was also championed by celebrities such as Taylor Swift, Bill Gates and Mark Zuckerberg. The challenge was started in a bid to raise awareness for the disease known as amyotrophic lateral sclerosis, and the use of social media spreading this challenge resulted in its resounding success. A more recent example would be the Trashtag Challenge, where social media users pick up litter, clean their environment and later post a photo of their accomplishments on social media. It has allowed users a better insight of the sheer scale of the plastics pollution problem. It is a good example of how social media can go further than raising awareness to mobilizing individuals to act, producing tangible results. In this way, social media allows for the creation of unity among people, for it raises awareness about issues that society is confronted with and mobilizes people to act in the interest of their communities.

In essence, while social media may cause rifts to occur and widen within society, its tendency to unite and bring people together is even greater. The availability of information online allows for the forging of a more understanding and hence, more united world and is something that would have been unattainable in a world fragmented into isolated groups.""",
    teacher,
    "“Social Media divides more than it unites.” To what extent is this true?"
)

,text,category,start,end
0,It all started with Six Degrees. Launched in 1...,Claim,0,7058
0,It all started with Six Degrees. Launched in 1...,Claim,7058,14116
0,It all started with Six Degrees. Launched in 1...,Claim,14116,21174
0,It all started with Six Degrees. Launched in 1...,Claim,21174,28232
0,It all started with Six Degrees. Launched in 1...,Claim,28232,35290
0,It all started with Six Degrees. Launched in 1...,Claim,35290,42348
0,It all started with Six Degrees. Launched in 1...,Claim,42348,49406
0,It all started with Six Degrees. Launched in 1...,Claim,49406,56464
0,It all started with Six Degrees. Launched in 1...,Claim,56464,63522
0,It all started with Six Degrees. Launched in 1...,Claim,63522,70580
